In [1]:
from real_datasets import BreastCancerDataset, WineQualityRedDataset, WineQualityWhiteDataset, SouthGermanCreditDataset, CropMappingDataset

In [2]:
import numpy as np


In [3]:
# seeds = [
#     85287339, # 412
#     20200621154912, # 378
#     20200623170005 # 433
# ]
# np.random.seed(seeds[0])

In [4]:
from synthetic_datasets import ClusterDataset, TwoClusterDataset, DiffusedBenchmark, PrismDataset, TruncatedNormalPrism

In [5]:
from solvers import gurobi_solver, cplex_solver, separating_hyperplane
from constants import epsilon_P, epsilon_N, epsilon_R

In [6]:
datasets = {
    "Breast Cancer": BreastCancerDataset(),
    "Crop Mapping": CropMappingDataset(),
    "Wine Quality Red": WineQualityRedDataset(),
    "Wine Quality White": WineQualityWhiteDataset(),
    "South German Credit": SouthGermanCreditDataset(),
    "Cluster 8": ClusterDataset(d=8),
    "Two Cluster 8": TwoClusterDataset(d=8),
    "Cluster": ClusterDataset(d=11),
    "Two Cluster": TwoClusterDataset(d=11),
    "Diffused Benchmark": DiffusedBenchmark(),
    "Prism": PrismDataset(),
    "Truncated Normal Prism": TruncatedNormalPrism(),
}

In [7]:
times = 1
import numpy as np

In [8]:
results = {}

In [ ]:
for dataset_name, dataset in datasets.items():
    P, N = dataset.generate()
    t0, t1, t, l = dataset.params()
    # print(t1)
    final_res = []
    for i in range(times):
        # print(i)
        # print(l)
        # w, c, r = separating_hyperplane(P, N, epsilon_P, epsilon_N, epsilon_R, t, l, num_trials=10000)
        # # print(c) 
        # reach = np.dot(P, w) - c >= epsilon_P
        # print(np.sum(reach))
        # # xs ≤ 1 + sT w − c − P
        res_cplex = cplex_solver(theta=t, theta0=t0, theta1=t1, P=P, N=N, lambda_param=l)
        print("cplex", dataset_name, res_cplex['Reach'])
        res_gurobi = gurobi_solver(theta=t, theta0=t0, theta1=t1, P=P, N=N, lambda_param=l)
        print("gurobi", dataset_name, res_gurobi['Reach'])
        # print(res_gurobi)
        # final_res.append([res_gurobi, res_cplex])
        # print([res_gurobi['Reach'], res_cplex['Reach']])
        # print(r)
    results[dataset_name] = final_res
    

Set parameter LicenseID to value 2604237
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Breast Cancer 357.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Crop Mapping 2401.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Wine Quality Red 18.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Wine Quality White 73.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi South German Credit 172.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Cluster 8 44.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Two Cluster 8 25.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Cluster 47.0
Set parameter LicenseID to value 2604237
Set parameter TimeLimit to value 120
gurobi Two Cluster 25.0
Set parameter Lice

In [10]:
for name, res in results.items():
    # print(res)
    for i in range(times):
        print(res[i][0]['Reach'], res[i][1]['Reach'])

IndexError: list index out of range

In [11]:
import pandas as pd

In [12]:
new_ar = {"dataset": [], "gurobi_reach": [], "gurobi_nodes": [], "gurobi_time": [], "cplex_reach": [], "cplex_nodes": [], "cplex_time": [],}

In [13]:
for name, res in results.items():
    for subres in res:
        res_g = subres[0]
        res_c = subres[1]
        new_ar['dataset'].append(name)
        new_ar['gurobi_reach'].append(res_g['Reach'])
        new_ar['gurobi_nodes'].append(res_g['Node Count'])
        new_ar['gurobi_time'].append(res_g['Time taken'])
        new_ar['cplex_nodes'].append(res_c['Node Count'])
        new_ar['cplex_reach'].append(res_c['Reach'])
        new_ar['cplex_time'].append(res_c['Time taken'])

In [14]:
# new_ar

In [15]:
n = pd.DataFrame(new_ar)

In [16]:
n.to_csv("res2.csv", index=None)

In [17]:
# df1 = pd.read_csv("./res.csv")
df2 = pd.read_csv("./res2.csv")

In [18]:
combined = pd.concat([ df2])

In [19]:
combined.to_csv("fin_res.csv", index=None)

In [20]:
avg_d = {} 
avg_d['dataset'] = [] 
avg_d['avg_gurobi_reach'] = []
avg_d['avg_gurobi_nodes'] = []
avg_d['avg_gurobi_time'] = []
avg_d['avg_cplex_reach'] = []
avg_d['avg_cplex_nodes'] = []
avg_d['avg_cplex_time'] = []

In [21]:
for name, dataset in datasets.items():
    cnt = 0 
    avg_gurobi_reach = 0 
    avg_gurobi_nodes = 0
    avg_gurobi_time = 0
    avg_cplex_reach = 0
    avg_cplex_nodes = 0
    avg_cplex_time = 0
    for idx, row in combined[combined['dataset'] == name].iterrows():
        cnt += 1
        avg_gurobi_reach += row['gurobi_reach']
        avg_gurobi_nodes += row['gurobi_nodes']
        avg_gurobi_time += row['gurobi_time']
        avg_cplex_reach += row['cplex_reach']
        avg_cplex_nodes += row['cplex_nodes']
        avg_cplex_time += row['cplex_time']
    avg_gurobi_reach /= cnt
    avg_gurobi_nodes /= cnt
    avg_gurobi_time /= cnt
    avg_cplex_reach /= cnt
    avg_cplex_nodes /= cnt
    avg_cplex_time /= cnt
    avg_d['dataset'].append(name)
    avg_d['avg_gurobi_reach'].append(avg_gurobi_reach)
    avg_d['avg_gurobi_nodes'].append(avg_gurobi_nodes)
    avg_d['avg_gurobi_time'] .append(avg_gurobi_time)
    avg_d['avg_cplex_reach'].append(avg_cplex_reach)
    avg_d['avg_cplex_nodes'].append(avg_cplex_nodes)
    avg_d['avg_cplex_time'].append(avg_cplex_time)
    
         

In [22]:
avg_df = pd.DataFrame(avg_d)

In [23]:
avg_df.to_csv("avg_df.csv", index=None)